In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import datasets, preprocessing, metrics
# import libraries
import sklearn as sk
import math 
import pprint

In [4]:
df = pd.read_json('companies.json')
df.head(5)

,vat,name,address,zipcode,city,cityname,protected,phone,email,fax,...,industrydesc,companycode,companydesc,creditstartdate,creditbankrupt,creditstatus,owners,productionunits,t,version
0,26911745,KMD A/S,Lautrupparken 40,2750,Ballerup,NaN,False,44601000.0,info@kmd.dk,44604106.0,...,Konsulentbistand vedrørende informationsteknologi,60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1017630322, 'main': False, 'name': 'K...",100,6
1,13594376,"PFA PENSION, FORSIKRINGSAKTIESELSKAB.",Sundkrogsgade 4,2100,København Ø,NaN,False,39175000.0,None,NaN,...,Livsforsikring,60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1000591581, 'main': False, 'name': 'P...",100,6
2,40916458,Dynatest A/S,Tempovej 27,2750,Ballerup,NaN,False,70253355.0,None,NaN,...,"Fremstilling af udstyr til måling, afprøvning,...",60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1025210030, 'main': True, 'name': 'Dy...",100,6
3,42334219,OTICON A/S,Kongebakken 9,2765,Smørum,NaN,False,39177100.0,None,39277900.0,...,Fremstilling af høreapparater og dele hertil,60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1006552453, 'main': False, 'name': 'O...",100,6
4,78040017,TOPDANMARK A/S,Borupvang 4,2750,Ballerup,NaN,False,70158585.0,None,NaN,...,Finansielle holdingselskaber,60,Aktieselskab,NaN,False,NaN,[{'name': 'Sampo plc'}],"[{'pno': 1015253246, 'main': False, 'name': 'T...",100,6


In [5]:
all = set()

for index, row in df.iterrows():
    words = row['industrydesc'].split()
    label = row['name']
    all.update(words)

In [6]:
print(all)

{'informationsteknologi', 'hertil', 'holdingselskaber', 'kontrol', 'Computerprogrammering', 'Fastnetbaseret', 'farmaceutiske', 'af', 'høreapparater', 'Konsulentbistand', 'måling,', 'Livsforsikring', 'og', 'navigation', 'afprøvning,', 'dele', 'bøger', 'telekommunikation', 'Finansielle', 'udstyr', 'vedrørende', 'Fremstilling', 'præparater', 'til', 'Udgivelse'}


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Bruger TfidfVectorizer til at transformere "activity" text til vectorer
vectorizer = TfidfVectorizer()
activity_vectors = vectorizer.fit_transform(df['industrydesc'])


# beregner den parvise cosinus-lighed mellem "activity vectors"
similarity_matrix = cosine_similarity(activity_vectors)

# Finder de gange hvor to firmaer har flest ligheder
max_sim = 0
max_i = 0
max_j = 0
for i in range(similarity_matrix.shape[0]):
    for j in range(i+1, similarity_matrix.shape[0]):
        if similarity_matrix[i, j] > max_sim:
            max_sim = similarity_matrix[i, j]
            max_i = i
            max_j = j

# printer navnene på de to firmaer som har flest ligheder
print(df.loc[max_i, 'name'])
print(df.loc[max_j, 'name'])

KMD A/S
Netcompany A/S


In [13]:
print(activity_vectors )

  (0, 12)	0.5773502691896258
  (0, 24)	0.5773502691896258
  (0, 13)	0.5773502691896258
  (1, 15)	1.0
  (2, 14)	0.3585920422630752
  (2, 18)	0.30796301135266807
  (2, 17)	0.3585920422630752
  (2, 1)	0.3585920422630752
  (2, 16)	0.3585920422630752
  (2, 21)	0.3585920422630752
  (2, 23)	0.3585920422630752
  (2, 0)	0.2441779759000562
  (2, 8)	0.27204114225155235
  (3, 9)	0.45754444018892443
  (3, 4)	0.45754444018892443
  (3, 11)	0.45754444018892443
  (3, 18)	0.39294448013678435
  (3, 0)	0.31155815557025845
  (3, 8)	0.34711007905893804
  (4, 10)	0.7071067811865475
  (4, 7)	0.7071067811865475
  (5, 3)	1.0
  (6, 12)	0.5773502691896258
  (6, 24)	0.5773502691896258
  (6, 13)	0.5773502691896258
  (7, 19)	0.5736145549596333
  (7, 5)	0.5736145549596333
  (7, 0)	0.39059439270573415
  (7, 8)	0.4351651468853661
  (8, 3)	1.0
  (9, 12)	0.5773502691896258
  (9, 24)	0.5773502691896258
  (9, 13)	0.5773502691896258
  (10, 2)	0.6371012829932499
  (10, 22)	0.6371012829932499
  (10, 0)	0.433824746201401
  (11

In [15]:
# Extend it with a column containing the web addresses of the companies. Scrap
# information from the web pages, read the text paragraphs only

#Add the company websites to the dataframe

company_websites = ['https://www.kmd.dk/','https://pfa.dk/privat/', 'https://dynatest.com/', 'https://www.oticon.dk/hearing-aid-users','https://www.topdanmark.dk/','https://northtech.dk/','https://www.netcompany.com/da','https://www.novonordisk.dk/','https://www.formpipe.com/da/','https://www.alpha-solutions.com','https://dafolo.dk/', 'https://nuuday.dk/']

In [17]:
df = df.assign(website=company_websites)

df.head(20)

,vat,name,address,zipcode,city,cityname,protected,phone,email,fax,...,companycode,companydesc,creditstartdate,creditbankrupt,creditstatus,owners,productionunits,t,version,website
0,26911745,KMD A/S,Lautrupparken 40,2750,Ballerup,NaN,False,44601000.0,info@kmd.dk,44604106.0,...,60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1017630322, 'main': False, 'name': 'K...",100,6,https://www.kmd.dk/
1,13594376,"PFA PENSION, FORSIKRINGSAKTIESELSKAB.",Sundkrogsgade 4,2100,København Ø,NaN,False,39175000.0,None,NaN,...,60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1000591581, 'main': False, 'name': 'P...",100,6,https://pfa.dk/privat/
2,40916458,Dynatest A/S,Tempovej 27,2750,Ballerup,NaN,False,70253355.0,None,NaN,...,60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1025210030, 'main': True, 'name': 'Dy...",100,6,https://dynatest.com/
3,42334219,OTICON A/S,Kongebakken 9,2765,Smørum,NaN,False,39177100.0,None,39277900.0,...,60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1006552453, 'main': False, 'name': 'O...",100,6,https://www.oticon.dk/hearing-aid-users
4,78040017,TOPDANMARK A/S,Borupvang 4,2750,Ballerup,NaN,False,70158585.0,None,NaN,...,60,Aktieselskab,NaN,False,NaN,[{'name': 'Sampo plc'}],"[{'pno': 1015253246, 'main': False, 'name': 'T...",100,6,https://www.topdanmark.dk/
5,10121361,NORTHTECH ApS,"Wildersgade 51, 1",1408,København K,NaN,True,NaN,kontakt@northtech.dk,NaN,...,80,Anpartsselskab,NaN,False,NaN,None,"[{'pno': 1009658463, 'main': True, 'name': 'NO...",100,6,https://northtech.dk/
6,14814833,Netcompany A/S,"Grønningen 17, 1",1270,København K,NaN,True,70131440.0,info@netcompany.com,NaN,...,60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1017633488, 'main': False, 'name': 'N...",100,6,https://www.netcompany.com/da
7,24256790,NOVO NORDISK A/S,Novo Alle 1,2880,Bagsværd,NaN,False,44448888.0,None,NaN,...,60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1017661031, 'main': False, 'name': 'N...",100,6,https://www.novonordisk.dk/
8,29177015,FORMPIPE SOFTWARE A/S,Lautrupvang 1,2750,Ballerup,NaN,False,33256555.0,None,NaN,...,60,Aktieselskab,NaN,False,NaN,[{'name': 'Formpipe Software AB'}],"[{'pno': 1014492123, 'main': False, 'name': 'F...",100,6,https://www.formpipe.com/da/
9,27506488,ALPHA SOLUTIONS A/S,"Gothersgade 14, 4",1123,København K,NaN,True,70206538.0,None,NaN,...,60,Aktieselskab,NaN,False,NaN,None,"[{'pno': 1010322290, 'main': True, 'name': 'AL...",100,6,https://www.alpha-solutions.com


In [18]:

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [21]:
new_df = pd.DataFrame(columns=['website', 'title', 'description'])

# for loop til at gå igennem df og få alle website values ud
for index, row in df.iterrows():
    website_url = row['website']
    # use request til at få website content ud
    res = requests.get(website_url)
    # bruger BeautifulSoup til at extract website data
    soup = BeautifulSoup(res.content, 'html.parser')
    # extract title og description
    title = soup.title.text if soup.title else None
    description = soup.find('meta', attrs={'name': 'description'})['content'] if soup.find('meta', attrs={'name': 'description'}) else None
    # laver ny row med det scrapede data
    new_row = pd.DataFrame({'website': [website_url], 'title': [title], 'description': [description]})
    # append ny row til ny dataframe(måske bruge concat istedet??)
    new_df = new_df.append(new_row, ignore_index=True)

# print new dataframe
new_df.head(20)

/var/folders/f4/g09hx1n5747dttb_p424s85w0000gn/T/ipykernel_3293/4209860410.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)


,website,title,description
0,https://www.kmd.dk/,KMD - IT og teknologi til det offentlige og er...,KMD er blandt nogle af Danmarks største it- og...
1,https://pfa.dk/privat/,Pension – få størst mulig udbytte af dine pens...,None
2,https://dynatest.com/,Dynatest - Home,Dynatest provides the best pavement testing eq...
3,https://www.oticon.dk/hearing-aid-users,\r\n\tOticons høreapparater | Livsændrende hør...,Oticons høreapparater kan hjælpe dig med at hø...
4,https://www.topdanmark.dk/,Topdanmark | Vi er her for at hjælpe,Hos os skal du ikke bekymre dig om dine forsik...
5,https://northtech.dk/,NorthTech | IT med omtanke,None
6,https://www.netcompany.com/da,Netcompany - IT and Business Consulting,Netcompany leverer forretningskritiske it-løsn...
7,https://www.novonordisk.dk/,Velkommen til Novo Nordisk Danmark,"Vi er en global lægemiddelvirksomhed, der blev..."
8,https://www.formpipe.com/da/,Formpipe - En værdifuld forbindelse til jeres ...,Vi har over 5500 kunder på tværs af brancher o...
9,https://www.alpha-solutions.com,None,None


In [36]:
#Gemmer til csv
new_df.to_csv("companyWithDescripton.csv")

In [35]:
## bruger csvfilen til at arbejde med
finaldf = pd.read_csv("companyWithDescripton.csv")
#Fjerner første column da den ikke skulle være der
finaldf = finaldf.iloc[: , 1:]
finaldf.head()

,website,title,description
0,https://www.kmd.dk/,KMD - IT og teknologi til det offentlige og er...,KMD er blandt nogle af Danmarks største it- og...
1,https://pfa.dk/privat/,Pension – få størst mulig udbytte af dine pens...,NaN
2,https://dynatest.com/,Dynatest - Home,Dynatest provides the best pavement testing eq...
3,https://www.oticon.dk/hearing-aid-users,\r\n\tOticons høreapparater | Livsændrende hør...,Oticons høreapparater kan hjælpe dig med at hø...
4,https://www.topdanmark.dk/,Topdanmark | Vi er her for at hjælpe,Hos os skal du ikke bekymre dig om dine forsik...


In [1]:
from sentence_transformers import SentenceTransformer, util

In [2]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [36]:
#my_list = list(all)
# Create an empty list to hold the descriptions
description_list = []

# Loop gennem  "description" column 
for description in finaldf["description"]:
    # Append description til listen
    description_list.append(description)

print(description_list)

['KMD er blandt nogle af Danmarks største it- og softwarevirksomheder. KMD leverer it-løsninger til erhverv og det offentlige. Se mere her.', nan, 'Dynatest provides the best pavement testing equipment, software, and technical support. Get accurate data and the highest standard of pavement technology.', 'Oticons høreapparater kan hjælpe dig med at høre bedre. Se vores høreløsninger og trådløse tilbehør og læs mere om høretab og tinnitus.', 'Hos os skal du ikke bekymre dig om dine forsikringer. Vi gør det nemt og enkelt. Og hjælper dig, når uheldet er ude.', nan, 'Netcompany leverer forretningskritiske it-løsninger og rådgivning, der hjælper vores kunder med at opnå markante forretningsfordele og effektiviseringer ved udnyttelse af den nyeste teknologi', 'Vi er en global lægemiddelvirksomhed, der blev grundlagt i 1923 med hovedkontor lige uden for København. Vores formål er at drive forandring for at bekæmpe diabetes og andre alvorlige kroniske sygdomme som svær overvægt og sjældne blod

In [37]:
# Create embeddings from texts
corpus_embeddings = embedder.encode(description_list)
corpus_embeddings

array([[-0.07696682,  0.09572213,  0.05159627, ...,  0.02283243,
         0.00255517,  0.09351519],
       [-0.06429786, -0.02824019,  0.04525843, ...,  0.00623698,
         0.01682923, -0.03065147],
       [-0.08750798, -0.03086364,  0.02757041, ...,  0.01552462,
         0.03092369, -0.02608698],
       ...,
       [-0.06429786, -0.02824019,  0.04525843, ...,  0.00623698,
         0.01682923, -0.03065147],
       [-0.06429786, -0.02824019,  0.04525843, ...,  0.00623698,
         0.01682923, -0.03065147],
       [-0.02732973, -0.03825023, -0.04211733, ...,  0.0500284 ,
         0.06102842, -0.00090102]], dtype=float32)

In [38]:
query = ["Bedste bøger?"]

In [39]:
query_embedding = embedder.encode(query)
query_embedding

array([[-9.41642001e-03,  6.15102686e-02,  9.94289177e-04,
        -7.33175352e-02,  9.07757506e-03,  5.40697649e-02,
         3.70966792e-02,  8.53809565e-02,  6.30769879e-02,
         3.23724048e-03,  3.66597273e-03, -4.71462533e-02,
         3.48847322e-02, -6.39309362e-02, -3.12425848e-02,
        -2.08076034e-02, -5.55289537e-02,  6.18478544e-02,
         4.08132114e-02,  2.06734110e-02, -5.93497120e-02,
         6.47513121e-02,  5.07715121e-02, -5.70421759e-03,
         5.96180512e-03, -1.72668938e-02,  7.52001181e-02,
        -7.59927835e-03, -2.81740334e-02, -7.84099326e-02,
         8.19818228e-02, -2.64089219e-02, -1.57340989e-02,
        -3.34676355e-02,  7.86429644e-03,  3.35057415e-02,
        -1.13928979e-02, -5.91039732e-02, -1.48716625e-02,
         8.02243203e-02, -9.99341607e-02, -3.81617583e-02,
        -4.31144126e-02, -1.09819241e-01,  2.86538675e-02,
         1.42751008e-01, -4.75820489e-02, -2.30991039e-02,
        -2.03685388e-02,  1.29586142e-02, -1.25982026e-0

In [40]:
# Test
# Compute similarities
for embedding in corpus_embeddings:
    sim = util.cos_sim(query_embedding, embedding)
    print("{0:.4f}".format(sim.tolist()[0][0]))

0.5438
0.1014
0.0424
0.5754
0.5520
0.1014
0.4546
0.5456
0.5010
0.1014
0.1014
0.1142


In [41]:
queries = ["Teknologi til det offentlige", "Pensions udbytte af penge", "Hjælp med forsikring",]

In [42]:
query_embedding = embedder.encode(queries)
query_embedding

array([[-0.0334284 ,  0.13421336, -0.03188007, ...,  0.04033926,
         0.00260819, -0.04328682],
       [-0.03288651,  0.03153046, -0.06571053, ...,  0.01602945,
         0.04462356,  0.02888331],
       [-0.01592072,  0.13379307, -0.05426133, ...,  0.00648319,
         0.01642444,  0.04259035]], dtype=float32)

In [43]:
import scipy.spatial

for query, query_embedding in zip(queries, query_embedding):
    # Convert query_embedding to a 1-dimensional array
    query_embedding = np.array(query_embedding)

    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n======================\n")
    print("Query:", query)
    print("\nTop 3 most similar sentences in corpus:\n")

    for idx, distance in results[0:closest_n]:
        print(description_list[idx].strip(), "(Score: %.4f)" % (1-distance))



Query: Teknologi til det offentlige

Top 3 most similar sentences in corpus:

KMD er blandt nogle af Danmarks største it- og softwarevirksomheder. KMD leverer it-løsninger til erhverv og det offentlige. Se mere her. (Score: 0.4041)
Hos os skal du ikke bekymre dig om dine forsikringer. Vi gør det nemt og enkelt. Og hjælper dig, når uheldet er ude. (Score: 0.4011)
Netcompany leverer forretningskritiske it-løsninger og rådgivning, der hjælper vores kunder med at opnå markante forretningsfordele og effektiviseringer ved udnyttelse af den nyeste teknologi (Score: 0.3992)


Query: Pensions udbytte af penge

Top 3 most similar sentences in corpus:

Vi er en global lægemiddelvirksomhed, der blev grundlagt i 1923 med hovedkontor lige uden for København. Vores formål er at drive forandring for at bekæmpe diabetes og andre alvorlige kroniske sygdomme som svær overvægt og sjældne blodsygdomme og endokrine sygdomme. (Score: 0.3254)
Vi har over 5500 kunder på tværs af brancher over hele verden, hv